In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "CY"
BARK_VAR = "CA"
FIA_SPCD = 805

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(-0.19128, 0.96147, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,2969.000000,2.969000e+03,2969.0,2969.000000,2969.000000,2969.000000,2969.000000,2969.000000,2969.000000,2969.00000,...,0.0,0.0,2969.000000,2969.000000,2969.000000,2969.000000,2969.0,2969.000000,2969.000000,2969.000000
mean,530.730886,1.169891e+11,805.0,61.068597,0.579906,3.313468,34.438195,8.904648,0.427252,0.41100,...,NaN,NaN,2005.089929,8.338498,2015.089929,7.203436,10.0,1.395419,10.809700,201.922870
std,39.081641,1.289452e+11,0.0,45.030845,0.175755,1.561794,12.598706,4.963381,0.168788,0.16657,...,NaN,NaN,2.449970,1.971222,2.449970,2.150748,0.0,1.748679,6.001782,113.896004
min,501.000000,6.010156e+10,805.0,10.000000,0.000000,0.000000,3.000000,5.000000,0.030000,0.01000,...,NaN,NaN,2001.000000,1.000000,2011.000000,3.000000,10.0,0.000000,0.000000,0.000000
25%,508.000000,6.021055e+10,805.0,10.000000,0.460000,2.181662,25.000000,5.900000,0.300000,0.30000,...,NaN,NaN,2003.000000,7.000000,2013.000000,5.000000,10.0,0.000000,7.000000,109.000000
50%,514.000000,6.030618e+10,805.0,60.221987,0.600000,3.368485,35.000000,7.300000,0.400000,0.40000,...,NaN,NaN,2005.000000,9.000000,2015.000000,7.000000,10.0,0.000000,13.000000,201.000000
75%,518.000000,6.050438e+10,805.0,102.500000,0.700000,4.363323,43.000000,9.700000,0.550000,0.50000,...,NaN,NaN,2007.000000,10.000000,2017.000000,9.000000,10.0,4.000000,16.000000,305.000000
max,712.000000,4.102034e+11,805.0,284.711358,1.410000,6.283185,81.000000,42.900000,0.950000,0.99000,...,NaN,NaN,2009.000000,12.000000,2019.000000,12.000000,10.0,4.000000,19.000000,397.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [14:01<00:00,  2.38it/s, 255 steps of size 2.28e-02. a


Done.
Samples saved at ../../models/maicf/CY//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/CY//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [26:51<00:00,  1.24it/s, 255 steps of size 1.43e-02. a


Done.
Samples saved at ../../models/maicf/CY//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/CY//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/CY//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/CY//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [15:13<00:00,  2.19it/s, 255 steps of size 1.45e-02. a


Done.
Samples saved at ../../models/maicf/CY//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/CY//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [26:59<00:00,  1.24it/s, 255 steps of size 1.79e-02. a


Done.
Samples saved at ../../models/maicf/CY//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/CY//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [11:11<00:00,  2.98it/s, 255 steps of size 2.25e-02. a


Done.
Samples saved at ../../models/maicf/CY//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/CY//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [11:01<00:00,  3.02it/s, 255 steps of size 2.07e-02. a


Done.
Samples saved at ../../models/maicf/CY//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/CY//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/CY//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/CY//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [19:26<00:00,  1.71it/s, 255 steps of size 1.99e-02. a


Done.
Samples saved at ../../models/maicf/CY//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/CY//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [19:32<00:00,  1.71it/s, 255 steps of size 1.79e-02. a


Done.
Samples saved at ../../models/maicf/CY//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/CY//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
